In [1]:
import os
import pandas as pd

In [2]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
DATA_PATH = ROOT_PATH + '/data'

In [3]:
# Carga puntos de elementos destacados

def LoadPointsExcel(layer):
    
    # Leemos el fichero de puntos
    df_points = pd.read_excel (DATA_PATH + '/layer-' + layer + '/points.xls', sheet_name='points')
    df_points['CodigoCultivo'].fillna('', inplace=True)
    df_points['CodigoPropietario'].fillna('', inplace=True)

    # Cargamos los ficheros de configuración
    df_c = pd.read_csv(CONFIG_PATH + '/cultivos.cfg', sep=',', encoding='UTF-8', na_filter= False)
    df_p = pd.read_csv(CONFIG_PATH + '/propietarios.cfg', sep=',', encoding='UTF-8', na_filter= False)

    # Completamos la información de los puntos
    df_points = pd.merge(df_points, df_c, on=['CodigoCultivo'], how='left')
    df_points = pd.merge(df_points, df_p, on=['CodigoPropietario'], how='left')
    df_points['Origen'] = 'layer-' + layer
    
    # Y adaptamos el dataset al formato requerido
    df_points.fillna('', inplace=True)
    df_points.drop(['CodigoCultivo', 'CodigoPropietario'], axis=1, inplace=True)
    df_points.rename(columns={'Latitud':'Latitude', 'Longitud':'Longitude'}, inplace=True)

    return df_points


In [4]:
# Función que guarda el dataset para ser visualizado en Google Maps

def SaveDataset(layer, df):

    s_file = 'dataset-points.pkl'
    s_path = DATA_PATH + '/layer-' + layer
    df.to_pickle(s_path + '/' + s_file)
    

In [5]:
# Función que fusiona la información del Catastro con la propia de la empresa 
# para generar el dataset que se visualizará postariormente

def GenerateDatasetPoints(layer):  
    s_path = DATA_PATH + '/layer-' + layer
    
    if (os.path.isdir(s_path)) & (os.path.isfile(s_path + '/points.xls')):
        # Cargamos los points desde la Excel
        df = LoadPointsExcel(layer)

        # Volcamos a disco el dataset
        SaveDataset(layer, df)
        

In [6]:
# Función que genera los datasets individuales de puntos de interés de todas las capas

def GenerateAllDatasetsPoints():
    for root, dirs, files in os.walk(DATA_PATH, topdown=False):
        for file_name in files:
            s_folder = os.path.basename(root)
            if (file_name == 'points.xls') & (s_folder.split('-')[0] == 'layer'):
                if s_folder.split('-')[1] is not None:
                    GenerateDatasetPoints(s_folder.split('-')[1])
                    

## Ejecución

In [7]:
# Generación de todos los datasets de Markers
GenerateAllDatasetsPoints()

# Generación de un dataset de markers
#GenerateDatasetPuntos(layer='caza')
